In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
from pyDOE import lhs
import math

D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1ty

In [2]:
RandomSeed = 9
np.random.seed(RandomSeed)
tf.set_random_seed(RandomSeed)

In [3]:
class PtPINNsss:
    # Initialize the class
    def __init__(self, x, t, u, lb, ub, lbp,ubp, layers,weights_values,biases_values):
        
        X = np.concatenate([x, t], 1)

        self.X = X
        
        self.x = X[:,0:1]
        self.t = X[:,1:2]
        
        self.u = u      
        self.lb = lb
        self.ub = ub
        self.hsadasjd=0
        self.ubp = ubp
        self.lbp = lbp        
        # Initialize NNs
        self.layers = layers
        self.hh=[]
        self.hh1=[]      
        
        self.weights, self.biases = self.initialize_NN(layers)
        
        # Calculate the values and convert them to float32
        value_float64 = 0.3
        self.b = tf.Variable(value_float64, dtype=tf.float64)
        
        value_float64=  1.0/(math.cosh(5*0.5)-1)       
        
        self.a = tf.constant(value_float64, dtype=tf.float64)     
        
        
        
        self.weights_values = weights_values        
        self.biases_values = biases_values    

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))    

        self.x_f_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.x_lb_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_b_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        self.x_ub_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        
        self.x_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.u_tf = tf.placeholder(tf.float64, shape=[None, self.u.shape[1]])

        self.u_pred, _ ,self.uuuuuu = self.net_AC(self.x_tf, self.t_tf)
        self.u_lb_pred, self.ux_lb_pred,_ = self.net_AC(self.x_lb_tf, self.t_b_tf)
        self.u_ub_pred, self.ux_ub_pred,_ = self.net_AC(self.x_ub_tf, self.t_b_tf)

        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)
        

        
        self.lossS = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))                                              
        self.lossB = tf.reduce_mean(tf.square(self.u_lb_pred - self.u_ub_pred)) + tf.reduce_mean(tf.square(self.ux_lb_pred - self.ux_ub_pred))
        self.lossfu =  tf.reduce_mean(tf.square(self.f_pred))    
        

        self.optimizer_Adam = tf.train.AdamOptimizer()

        self.loss  =  64 * self.lossS + self.lossB + self.lossfu
        
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)      

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    
        init = tf.global_variables_initializer()
        self.sess.run(init)
        self.save = tf.train.Saver(max_to_keep=1)
              
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = tf.Variable(tf.zeros([layers[l], layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    
    
    def initialize_NN2(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases      
    
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev, dtype=tf.float64))
    
    def custom_function(self,t):
        ab= self.b     
        condition2 = tf.less_equal(t, 0.2)
        condition3 = tf.less_equal(t, ab)
        condition4 = tf.less_equal(ab, 0.4) 
        BB=5*(t-0.2)
        B=1/(ab-0.2)*(t-0.2)
        
        return tf.where(condition2, tf.zeros_like(t), tf.where(condition4,tf.where(condition3, -2*B**3+3*B**2,tf.zeros_like(t)+1),-2*BB**3+3*BB**2))
    
 

    def input_encoding(self, t, x):    
        H=tf.concat([t,1+0*tf.cos(math.pi*x)],1)
        H=tf.concat([H,tf.cos(math.pi*x)],1)
        H=tf.concat([H,tf.sin(math.pi*x)],1) 
        H=tf.concat([H,tf.cos(2*math.pi*x)],1)
        H=tf.concat([H,tf.sin(2*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(3*math.pi*x)],1)
        H=tf.concat([H,tf.sin(3*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(4*math.pi*x)],1)
        H=tf.concat([H,tf.sin(4*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(5*math.pi*x)],1)
        H=tf.concat([H,tf.sin(5*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(6*math.pi*x)],1)
        H=tf.concat([H,tf.sin(6*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(7*math.pi*x)],1)
        H=tf.concat([H,tf.sin(7*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(8*math.pi*x)],1)
        H=tf.concat([H,tf.sin(8*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(9*math.pi*x)],1)
        H=tf.concat([H,tf.sin(9*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(10*math.pi*x)],1)
        H=tf.concat([H,tf.sin(10*math.pi*x)],1)           
        
        return H


    def neural_net(self, x,t, weights, biases,weights_values,biases_values):
        X = tf.concat([x,t],1)
        tt=5*t
        num_layers = len(weights) + 1
        H = self.input_encoding(tt, x)
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            W1 = weights_values[l]
            b1 = biases_values[l]   
            H0 = tf.add(tf.matmul(H, W), b)
            H1 = tf.add(tf.matmul(H, W1), b1)
            
            B = self.custom_function(t)   
            H0 = H0*B              
            
            H  = tf.add(H0,H1)
            H =  tf.tanh(H)
        
        W = weights[-1]
        b = biases[-1]
        W1 = weights_values[-1]
        b1 = biases_values[-1]  
        
        H0 = tf.add(tf.matmul(H, W), b)
        
        B = self.custom_function(t)   
        H0 = H0*B            
        
        H1 = tf.add(tf.matmul(H, W1), b1) 
        H  = tf.add(H0,H1)      
        Y = H
        return Y
    
    def net_AC(self, x, t):
        u = self.neural_net(x,t, self.weights, self.biases,self.weights_values, self.biases_values)
        u_x = tf.gradients(u, x)[0]
        u_t = tf.gradients(u, t)[0]

        return u, u_x, u_t
          
    

    def net_f(self, x, t):
        u, u_x, u_t = self.net_AC(x, t)
        
        u_xx = tf.gradients(u_x, x)[0]
        
        f_u = u_t - 0.0001*u_xx+5*u**3-5*u
        
        return f_u
    
    def callback(self, loss, a,b,lossfu, lossS, lossB):
        sss=self.hsadasjd
        if sss%1000==0:
            print('Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e ' % (loss, lossfu, lossS, lossB))
        sss=sss+1
        self.hsadasjd=sss 
        self.hh.append(a)      
        self.hh1.append(b)        
    def train(self, nIter, Nf, Nn, Nb):

        X_train = self.lbp + (self.ubp-self.lbp)*lhs(2, Nf)
        self.xtrain_f = X_train[:,0:1]
        self.ttrain_f = X_train[:,1:2] 
        
        X_lb_train = self.lbp + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_lb = X_lb_train[:,0:1]
        self.ttrain_b = X_lb_train[:,1:2]
        
        X_ub_train = [self.ubp[0],self.lbp[1]] + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_ub = X_ub_train[:,0:1]
        
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t, self.u_tf: self.u,
                   self.x_lb_tf: self.xtrain_lb, self.t_b_tf: self.ttrain_b, 
                   self.x_ub_tf: self.xtrain_ub, 
                   self.x_f_tf: self.xtrain_f, self.t_f_tf: self.ttrain_f}

        for it in range(nIter):
            loss_value = self.sess.run(self.loss, tf_dict)
            lossfu = self.sess.run(self.lossfu, tf_dict)
            lossS = self.sess.run(self.lossS, tf_dict)
            lossB = self.sess.run(self.lossB, tf_dict)
            self.sess.run(self.train_op_Adam, tf_dict)
                
            ab= self.sess.run(self.a)
            self.hh.append(ab)       
            abc= self.sess.run(self.b)
            self.hh1.append(abc)    

            print(abc)                    
            if it % 1000 == 0:
                print('It: %d, Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e' % (it, loss_value, lossfu, lossS, lossB))
                        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, method = 'L-BFGS-B', options = {'maxiter': 50000,'maxfun': 50000,'maxcor': 50,'maxls': 50,'ftol' : 1.0 * np.finfo(float).eps})                                                                                                         
        self.optimizer.minimize(self.sess, feed_dict = tf_dict, fetches = [self.loss,self.a,self.b,  self.lossfu, self.lossS, self.lossB], loss_callback = self.callback)        
                                    
    
    def predict(self, x, t):
        
        tf_dict = {self.x_tf: x, self.t_tf: t}
        u_star = self.sess.run(self.u_pred, tf_dict)
        u_starssss = self.sess.run(self.uuuuuu, tf_dict)        
        return u_star,u_starssss

    def saver(self, string):
        self.save.save(self.sess, 'ckpt/'+string)
        
        
    def sssss(self):
        return self.hh,  self.hh1           
        
        
    def restore(self):
        model_file = tf.train.latest_checkpoint('ckpt/')
        self.save.restore(self.sess, model_file)

In [4]:
import pickle

In [5]:
with open('bcweights.pkl', 'rb') as f:
    weights_values = pickle.load(f)

In [6]:
with open('bcweights1.pkl', 'rb') as f:
    biases_values = pickle.load(f)

In [7]:
if __name__ == "__main__": 
           
    
    # Doman bounds
    lb = np.array([-1.0, 0])
    ub = np.array([1.0, 0.2])

    # Pre-training interval
    lbp = np.array([-1.0, 0.2])    
    ubp = np.array([1.0,  0.4])


    
    layers = [22, 100, 100, 100, 100, 1]
    
    data = scipy.io.loadmat('AC.mat')
    
    t = data['tt'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = data['uu']
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]

    def IC(x):
        u = x**2*np.cos(np.pi*x)
        return u

    N0 = 1200
    x=np.linspace(-1,1,N0).flatten()[:,None]  
    X0 =x
    T0 = np.full((N0,1), lb[1])
    U0 = IC(X0)
      
    model2 = PtPINNsss(X0, T0, U0, lb, ub,lbp, ubp, layers,weights_values,biases_values)                        
    model2.train(5000, 4000, 1200, 240)  
    model2.saver('testmodel.ckpt')
    


    u_pred ,_= model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0.3
It: 0, Loss: 8.347872e-03, Lossfu: 8.346e-03, LossS: 2.295e-08, LossB: 1.602e-28
0.30074413433249153
0.30144305676531696
0.3021757758688378
0.3029754932575852
0.30381550241779504
0.3046607987503202
0.30548355781577924
0.30628829204016084
0.30709118829102044
0.30789415434003553
0.308682720266403
0.3094376028700986
0.31014769093412964
0.31081455291712834
0.3114467420430943
0.3120559500090954
0.31264912024806496
0.3132254918850682
0.3137791094201713
0.31430161223370257
0.3147879649580364
0.315238016545061
0.315656232125464
0.316047721350402
0.3164153088895342
0.31675983528791185
0.3170816504081815
0.31738194991876256
0.317662897139554
0.3179272873483652
0.31817788275578734
0.31841656654984407
0.3186443019588456
0.3188613445861387
0.31906804856571386
0.31926496700240886
0.3194528659548104
0.31963225532175255
0.31980338864477786
0.3199662919636434
0.3201208907098906
0.

0.32445673982240053
0.324465270620651
0.32447380264732906
0.324482338232603
0.3244908767602324
0.32449941766997503
0.32450796045133073
0.32451650464015624
0.3245250498134528
0.3245335955853753
0.3245421416028651
0.324550690500658
0.3245592416925163
0.3245677946454068
0.32457635350341035
0.3245849173683789
0.32459348542639804
0.3246020569397686
0.32461063123948425
0.32461920771822506
0.3246277858239881
0.32463636505371596
0.3246449478572525
0.3246535335285743
0.3246621214272225
0.3246707109722314
0.32467930163630254
0.32468789286822763
0.32469648423886865
0.32470507535709975
0.32471366586614453
0.324722255440464
0.3247308437829858
0.32473943133317296
0.32474801777216317
0.3247566028088897
0.3247651861771791
0.32477376763323595
0.32478234734025896
0.3247909250535776
0.3247995005492372
0.3248080736228285
0.3248166440861955
0.32482521189575736
0.32483377687954906
0.3248423388790174
0.3248508977477564
0.3248594533504611
0.32486800556271267
0.3248765542686335
0.324885099360668
0.324893640522

0.32784497249025557
0.3278519689773116
0.3278589608610027
0.3278659490825693
0.3278729331216345
0.32787991251007664
0.32788688682635964
0.3278938556910732
0.32790081876294236
0.32790777573549945
0.32791472633344476
0.32792167030966923
0.3279286084226577
0.32793554037399675
0.3279424658952402
0.32794938474524576
0.32795629670730125
0.3279632015869685
0.3279700992100255
0.32797698942001846
0.32798387207679564
0.32799074705459774
0.32799761424107576
0.3280044742751035
0.3280113269940552
0.3280181722519901
0.3280250099183126
0.32803183987590573
0.3280386620201445
0.3280454762573241
0.32805228233986267
0.3280590802097025
0.328065869815035
0.32807265111002953
0.32807942405413637
0.32808619263111444
0.32809295640935465
0.32809971500104157
0.32810646805747895
0.32811321526522286
0.32811995634224267
0.3281266910347063
0.3281334191147421
0.3281401413551264
0.3281468574781717
0.3281535672343469
0.3281602703991904
0.32816696677117074
0.3281736561694234
0.32818033843188155
0.3281870134134588
0.3281

0.3302753981961479
0.33028013402080597
0.3302848645699693
0.3302895897487115
0.33029430947249655
0.33029902366587593
0.33030373226167353
0.33030843520023345
0.33031313242882177
0.33031782390072867
0.33032251305166893
0.3303271994992468
0.3303318828996579
0.3303365629440211
0.33034123935501203
0.33034591188348533
0.3303505803058191
0.3303552444212307
0.33035990404967724
0.33036456599551367
0.3303692294155277
0.3303738935511585
0.33037855771968955
0.33038322130664227
0.3303878837592061
0.3303925445801721
0.3303972033226701
0.3304018595848049
0.33040651300516666
0.3304111632586559
0.3304158100533206
0.33042045312711854
0.33042509224513966
0.33042972719665115
0.33043435779245556
0.33043898714632625
0.33044361477910666
0.3304482402600951
0.33045286320204986
0.33045748325697866
0.33046210011213584
0.3304667134865309
0.33047132312808486
0.33047592881059085
0.33048053033122426
0.33048512750805525
0.3304897210868228
0.33049431083005854
0.33049889652474407
0.33050347797985546
0.33050805502404657

0.3319501327765841
0.3319532977016238
0.3319564592532377
0.3319596173768336
0.3319627720237749
0.3319659231507731
0.3319690707194363
0.33197221469578275
0.3319753550497849
0.33197849175488536
0.3319816247876916
0.33198475412772577
0.33198787975707716
0.33199100166014106
0.3319941223090361
0.33199724144201503
0.3320003588239588
0.3320034742435964
0.3320065875110233
0.33200969845571726
0.3320128069246323
0.3320159127804847
0.3320190159000606
0.33202211617285327
0.33202521349992836
0.3320283077927794
0.33203139897227946
0.33203448696760696
0.33203757171549747
0.3320406531595486
0.3320437312495159
0.33204680466054326
0.3320498734787876
0.3320529377824567
0.33205599764263716
0.33205905312393047
0.33206210428504973
0.3320651511793597
0.3320681938555322
0.3320712323579717
0.3320742667271537
0.3320772970000751
0.33208032321060915
0.33208334538982287
0.33208636356622445
0.33208937776604674
0.33209238801341473
0.3320953943305661
0.3320984007126499
0.3321014067801564
0.332104412191856
0.332107416

...

0.3457547679248647
0.34575650156940374
0.3457582311250996
0.3457599589533884
0.34576168134506796
0.3457634023139988
0.3457651189077724
0.3457668328937823
0.3457685436394418
0.34577025148608614
0.34577195572057434
0.345773657839325
0.3457753565306357
0.34577705230031613
0.3457787455343762
0.34578043536149644
0.3457821228786357
0.3457838075222595
0.3457854894601864
0.34578716862731207
0.34578884586694125
0.3457905196117672
0.345792191768745
0.3457938607132835
0.3457955278447966
0.34579719189305863
0.3457988543774868
0.34580051319214944
0.34580217110096956
0.3458038250341707
0.3458054781942411
0.34580712730383156
0.3458087760676363
0.3458104201063514
0.34581206476095316
0.34581370366971137
0.3458153443995714
0.3458169782862938
0.3458186154204556
0.34582024399843997
0.34582187840698947
0.3458235012624875
0.3458251339971745
0.3458267509892408
0.3458283839335561
0.34582999491973965
0.34583163181060134
0.34583323717187126
0.3458348868984569
0.3458364929926847
0.3458381729385587
0.345839790857

0.3490997343076191
0.3491028425272339
0.34910815594565436
0.34911476620092147
0.349128069591433
0.34914801135118745
0.3491889743804336
0.3492528617274535
0.3493705235199125
0.3495262508803867
0.34973367493729907
0.34994025210103685
0.35013188928956696
0.3503150330053756
0.35049424017465425
0.35069139916985537
0.35087397154265554
0.3510405003656833
0.35120965328056203
0.3513704984076797
0.3515241348731285
0.3516658399096724
0.3517981848634402
0.3519315688831609
0.35205252508872054
0.3521628225545145
0.3522747979973607
0.35237730847957227
0.35247061290919723
0.352562734230503
0.35264766507319995
0.35272616274855045
0.3528008241253211
0.3528697423608223
0.35293500213958545
0.3529952672458548
0.35305062271919285
0.35310455402012936
0.3531536053963797
0.35319832948996793
0.353242498937737
0.3532825282377644
0.353318865875914
0.35335467115794394
0.35338725476197963
0.35341691597619895
0.35344572243460287
0.3534719682838998
0.3534962831949496
0.3535195408317924
0.35354074901464366
0.353560857

0.3569212091825227
0.356924410073067
0.3569276078346726
0.3569308024474796
0.356933993881323
0.3569371821171545
0.3569403671370386
0.35694354892162605
0.35694672746261097
0.35694990274102145
0.3569530747421599
0.35695624354227923
0.35695940913346075
0.35696257149553656
0.35696573062251263
0.35696888650100683
0.35697203912519443
0.3569751884881115
0.35697833458338624
0.3569814774094182
0.35698461695957295
0.3569877532361828
0.3569908862331939
0.3569940159552058
0.35699714239748764
0.35700026556536074
0.35700338545608695
0.3570065020749718
0.35700961542169374
0.35701272550080154
0.3570158323142006
0.35701893586552474
0.3570220361588017
0.35702513304123157
0.35702822653410554
0.35703131665372645
0.3570344034196056
0.3570374868451705
0.3570405669482626
0.35704364374015635
0.3570467172370894
0.35704978744877075
0.35705285438993145
0.35705591806926146
0.3570589785001381
0.35706203569048445
0.3570650896524129
0.3570681410138588
0.3570711897232508
0.35707423573107294
0.3570772789966183
0.35708

0.3692908571741147
0.369302162862468
0.36931684013985533
0.36934373861356046
0.3693827711123653
0.3694539901858215
0.3695535298145204
0.3697085462493505
0.36988515912286013
0.37008036268844924
0.3702593783209891
0.3704224849538416
0.3705915626108407
0.37075740722238537
0.370925235002432
0.37107754312371166
0.37121788853605603
0.37136414088331116
0.37150050255375805
0.3716264596263118
0.37174700145640277
0.3718592377182825
0.37196932421921103
0.3720699288525803
0.3721617267632834
0.37225452361906763
0.37233919930177517
0.37241602126395484
0.3724922248677549
0.3725623501181466
0.3726272725613923
0.37268933848811364
0.37274593484634544
0.37279999004645464
0.37285042764928394
0.37289596833101046
0.37294034736693404
0.37298135433320906
0.37301837344533834
0.3730544767146201
0.3730877232957046
0.3731181100171528
0.37314766060983345
0.3731745325753417
0.37319957061779385
0.3732238305193957
0.3732456721586997
0.3732664499551414
0.3732863994236037
0.3733044121951843
0.37332199103094066
0.373338

randorm seed: 9
Training error in pre-training interval:(0.00,0.40)
Error2 u: 1.545699e+01
Error1 u: 5.265114e+00
Errorf u: 4.575606e+01


In [8]:
    t = data['tt'].flatten()[:,None][0:41] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,0:41] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,0.40)
Error2 u: 1.711667e-04
Error1 u: 5.102864e-05
Errorf u: 2.847895e-03


In [16]:
    t = data['tt'].flatten()[:,None][40:81] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,40:81] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,0.40)
Error2 u: 2.306446e-04
Error1 u: 9.407172e-05
Errorf u: 6.077087e-04


In [17]:
    scipy.io.savemat("solution2.mat", {'u': u_pred})

In [18]:
    t = data['tt'].flatten()[:,None][41:81] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,41:81] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,0.40)
Error2 u: 2.316002e-04
Error1 u: 9.469894e-05
Errorf u: 6.077087e-04


In [19]:
    scipy.io.savemat("solution22.mat", {'u': u_pred})

In [ ]:
    t = data['tt'].flatten()[:,None][40:81] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,40:81] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

In [14]:
weights_values2 = model2.sess.run(model2.weights)
biases_values2 = model2.sess.run(model2.biases)

In [15]:
import pickle
with open('2bcweights.pkl', 'wb') as f:
    pickle.dump(weights_values2, f)
with open('2bcweights1.pkl', 'wb') as f:
    pickle.dump(biases_values2, f)